In [1]:
import tensorflow as tf

In [10]:
class NaiveDense:
    def __init__(self, input_size, output_size, activation):
        self.activation = activation
        
        W_shape = (input_size, output_size)
        W_initial_values = tf.random.uniform(W_shape, minval=0, maxval = 1e-1)
        self.W = tf.Variable(W_initial_values)
        
        b_shape = (output_size)
        b_initial_values = tf.zeros(b_shape)
        self.b = tf.Variable(b_initial_values)
        
    def __call__(self, inputs):
        return self.activation(tf.matmul(inputs, self.W) + self.b)
    
    @property
    def weights(self):
        return (self.W, self.b)
        

In [12]:
class NaiveSequential:
    def __init__(self, layers):
        self.layers = layers
        
    def __call__(self, inputs):
        x = inputs
        for layer in self.layers:
            x = layer(x)
        return x
    
    @property
    def weights(self):
        weights = []
        for layer in self.layers:
            weights += layer.weights
        return weights

In [13]:
Dense = NaiveDense(10,1, tf.nn.relu)

In [19]:
model = NaiveSequential([
    NaiveDense(input_size = 28 * 28, output_size = 512 , activation= tf.nn.relu),
    NaiveDense(input_size = 512, output_size = 10, activation = tf.nn.softmax)
])

assert len(model.weights) == 4

In [20]:
import math

In [22]:
class BatchGenerator:
    def __init__(self, images, labels, batch_size = 128):
        assert len(images) == len(labels)
        self.images = images
        self.labels = labels
        self.batch_size = batch_size 
        self.index = 0
        self.num_batches = math.ceil(len(images) / self.batch_size)
        
        
    def next(self):
        images = self.images[self.index : self.index + self.batch_size]
        labels = self.labels[self.index : self.index + self.batch_size]
        self. index += self.batch_size 
        return images , labels

In [39]:
def one_training_step(model, images_batch , labels_batch):
    with tf.GradientTape() as tape:
        predictions = model(images_batch)
        per_sample_loss = tf.keras.losses.sparse_categorical_crossentropy(labels_batch, predictions)
        average_loss = tf.reduce_mean(per_sample_loss)
    gradients = tape.gradient(average_loss, model.weights)
    update_weights(gradients, model.weights)
    return average_loss

In [24]:
learning_rate = 1e-3

In [27]:
def update_weights(gradients, weights):
    for g,w in zip(gradients, weights):
        w.assign_sub(g * learning_rate)

In [38]:
def fit(model, images, labels, epochs , batch_size = 128):
    for epoch_counter in range(epochs):
        print(f'Epoch  {epoch_counter}')
        batch_generator = BatchGenerator(images, labels)
               
        
        for batch_counter in range(batch_generator.num_batches):
            images_batch , labels_batch = batch_generator.next()
            loss = one_training_step(model, images_batch, labels_batch)

            if batch_counter % 100 == 0:
                print(f'Loss at batch {batch_counter} is {loss:.2f}')

In [34]:
from tensorflow.keras.datasets import mnist

In [30]:
(train_images , train_labels) , (test_images , test_labels) = mnist.load_data()


In [31]:
train_images = train_images.reshape((60000, 28 * 28)).astype('float32') / 255
test_images = test_images.reshape((10000, 28 * 28)).astype('float32') /255

In [40]:
fit(model, train_images, train_labels, epochs = 10, batch_size = 128)

Epoch  0
Loss at batch 0 is 3.89
Loss at batch 100 is 2.24
Loss at batch 200 is 2.22
Loss at batch 300 is 2.06
Loss at batch 400 is 2.25
Epoch  1
Loss at batch 0 is 1.89
Loss at batch 100 is 1.88
Loss at batch 200 is 1.85
Loss at batch 300 is 1.68
Loss at batch 400 is 1.87
Epoch  2
Loss at batch 0 is 1.57
Loss at batch 100 is 1.58
Loss at batch 200 is 1.52
Loss at batch 300 is 1.40
Loss at batch 400 is 1.54
Epoch  3
Loss at batch 0 is 1.32
Loss at batch 100 is 1.35
Loss at batch 200 is 1.25
Loss at batch 300 is 1.18
Loss at batch 400 is 1.30
Epoch  4
Loss at batch 0 is 1.12
Loss at batch 100 is 1.17
Loss at batch 200 is 1.05
Loss at batch 300 is 1.02
Loss at batch 400 is 1.13
Epoch  5
Loss at batch 0 is 0.98
Loss at batch 100 is 1.03
Loss at batch 200 is 0.91
Loss at batch 300 is 0.91
Loss at batch 400 is 1.00
Epoch  6
Loss at batch 0 is 0.87
Loss at batch 100 is 0.92
Loss at batch 200 is 0.81
Loss at batch 300 is 0.82
Loss at batch 400 is 0.91
Epoch  7
Loss at batch 0 is 0.79
Loss at 

In [42]:
import numpy as np

In [43]:
predictions = model(test_images)
predictions = predictions.numpy() 
predicted_labels = np.argmax(predictions, axis=1)
matches = predicted_labels == test_labels
print(f"accuracy: {matches.mean():.2f}")

accuracy: 0.81
